# LVV-T1782 DATA ANALISYS SCRIPT

The data collection for this analysis comes from the [LVV-T1782](https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T1782) v.1 for the Level3 and v.2 for TMA.
This analisys also wants to debug the M2 cell situation during the multiple *faults* happened during the tests at the TMA.
The fault and the interpretation of the results are discussed in the ticket [OBS-300](https://rubinobs.atlassian.net/browse/OBS-300)

In [ ]:
%matplotlib inline
import lsst_efd_client
from astropy.time import Time
from pandas import Timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib as mpl
import matplotlib.cm as cm
import tabulate
from lsst.ts.m2com.mock import MockModel
from lsst.ts.m2com.utility import correct_inclinometer_angle
from lsst.ts.m2com.constant import (LIMIT_FORCE_TANGENT_CLOSED_LOOP,
TANGENT_LINK_LOAD_BEARING_LINK, 
TANGENT_LINK_THETA_Z_MOMENT, 
TANGENT_LINK_TOTAL_WEIGHT_ERROR,
NUM_TANGENT_LINK,
NUM_ACTUATOR)
import pandas
import os
from sklearn.linear_model import HuberRegressor

In [ ]:
def add_force_error(data: pandas.DataFrame) -> pandas.DataFrame:
    """Fucntion to calculate the Tangent Force error if it is not present for the time widnows of the query.

    Args:
        data (pd.DataFrame): Input dataframe.

    Returns:
        pd.DataFrame: The input data with the tangent force error added.
    """

    df = data.copy()
    cols = [
        "force0",
        "force1",
        "force2",
        "force3",
        "force4",
        "force5",
        "weight",
        "sum",
    ]

    for i, row in enumerate(data.iterrows()):
        if abs(row[1]["inclinometerRaw"]) > 360:
            c = 1
            searching = True
            while searching:
                eval_angle = data.loc[data.index[i - c], "inclinometerRaw"]
                if abs(eval_angle) < 360:
                    angle = eval_angle
                    searching = False
                else:
                    c += 1
        else:
            angle = row[1]["inclinometerRaw"]

        mock = MockModel()
        angle = correct_inclinometer_angle(angle) #mock.control_open_loop.
        mock.control_open_loop.inclinometer_angle = angle
        tangent_force_error = mock._calculate_force_error_tangent(
            np.array(
                [
                    row[1]["measured0"],
                    row[1]["measured1"],
                    row[1]["measured2"],
                    row[1]["measured3"],
                    row[1]["measured4"],
                    row[1]["measured5"],
                ]
            )
        )

        df.loc[row[0], cols[:-2]] = tangent_force_error["force"]
        df.loc[row[0], cols[-2]] = tangent_force_error["weight"]
        df.loc[row[0], cols[-1]] = tangent_force_error["sum"]

    return df

In [ ]:
async def get_merge_query(
    efd_client: lsst_efd_client.EfdClient,
    topics: list,
    start_time: Time,
    end_time: Time,
    tolerance=None,
    direction: str = None,
) -> pd.DataFrame:
    """Function to merge multiple topics into a single dataframe.

    Args:
        efd_client (lsst_efd_client.EfdClient): EFD client object.
        topics (list): EFD topics to query and merge.
        start_time (Time): Start time of the query.
        end_time (Time): End time of the query.
        tolerance (float, optional): Tolerance time for matching, see pandas.merge_asof for more information. Defaults to None will use defualt value.
        direction (str, optional): Join method to pass to pd.merge_asod method. Defaults to None will use default value.

    Returns:
        pd.DataFrame: Merged dataframe.
    """

    query_df = list()
    calculate_force_error = False
    for topic in topics:
        topic_fields = await efd_client.get_fields(topic)
        query = efd_client.build_time_range_query(
            topic, topic_fields, start_time, end_time
        )
        data_query = await efd_client.influx_client.query(query)

        if len(data_query) == 0 and topic == "lsst.sal.MTM2.forceErrorTangent":
            print(f"{topic} is not present, try to calculate it.")
            calculate_force_error = True
        elif len(data_query) == 0:
            print(f"{topic} is not present.")
        else:
            query_df.append(data_query)

    if len(query_df) == 1:
        return query_df[0]
    elif len(query_df) == 0:
        print("No Dataframe retrieve")
        return query_df

    query_df.sort(key=lambda el: len(el), reverse=True)
    merge_df = query_df[0].copy()

    for i, df in enumerate(query_df[1:]):
        col_left = topics[i].split(".")[-1]
        col_right = topics[i + 1].split(".")[-1]

        if tolerance is None and direction is None:
            merge_df = lsst_efd_client.rendezvous_dataframes(
                merge_df,
                df,
                direction="nearest",
                suffixes=[f"_{col_left}", f"_{col_right}"],
            )

        elif tolerance is None and direction is not None:
            merge_df = lsst_efd_client.rendezvous_dataframes(
                merge_df,
                df,
                direction=direction,
                suffixes=[f"_{col_left}", f"_{col_right}"],
            )

        elif tolerance is not None and direction is None:
            merge_df = lsst_efd_client.rendezvous_dataframes(
                merge_df,
                df,
                direction="nearest",
                tolerance=tolerance,
                suffixes=[f"_{col_left}", f"_{col_right}"],
            )
        else:
            merge_df = lsst_efd_client.rendezvous_dataframes(
                merge_df,
                df,
                direction=direction,
                tolerance=tolerance,
                suffixes=[f"_{col_left}", f"_{col_right}"],
            )

    if calculate_force_error:
        merge_df = add_force_error(merge_df)
    return merge_df

## Tangetial analysis function

`plot_overview` generates a panel with a summary of the tangent link in terms of force and force error.
The panel is built as follows:
* Plot of the mirror walk
* Pair of subplot for each actuator showing the relation between _encoder position_-_measured force_ and _encoder position_-_force error_
* The last two plots are the _tangent force error sum_ and _tangent force error weight_ during the test.

In [ ]:
def plot_overview(
    query_df: pd.DataFrame,
    direction: str,
    prefix: str,
    title: str,
    out_path: str,
    huber: HuberRegressor,
    cmap: list,
) -> None:
    """Function to plot the overview of the data.

    Args:
        query_df (pd.DataFrame): Dataframe with the input data.
        direction (str): Direction of the movement (X, Y or Z).
        prefix (str): Prefix of the output file.
        title (str): Title of the plot.
        out_path (str): Output path.
        huber (HuberRegressor): Huber Regressor object.
        cmap (list): Color map.

    Returns:
        None
    """
    fig = plt.figure()
    fig.set_figheight(15)
    fig.set_figwidth(15)
    gs = GridSpec(5, 4)

    ax = fig.add_subplot(gs[0, :])
    x_time = np.arange(len(query_df.index))
    ax.scatter(x_time, query_df[direction], marker="+", color=cmap[0])
    ax.set_xlabel("TimeStamp")
    ax.set_ylabel(r"Movement, $\mu$m")
    ax.set_title(f"Movement Overview along the {direction}-axis")

    ticks = np.linspace(0, len(query_df.index) - 1, 5)
    ticks_label = [
        query_df.index[int(el)].strftime("%m-%dT%H:%M:%S") for el in ticks
    ]
    ax.set_xticks(ticks)
    ax.set_xticklabels(ticks_label)

    ax.grid()

    r = 1
    c = 0

    fit_df = query_df[[col for col in query_df.columns if "private" not in col]].copy().dropna()
    linear_fit_force_par = list()
    linear_fit_ferror_par = list()

    k = 1.0
    for i in range(0, NUM_TANGENT_LINK, 2):
        x = query_df[f"position{i}"]

        xmin, xmax = x.min(), x.max()

        fit_x = fit_df[f"position{i}"]

        A = np.vstack([fit_x, np.zeros(len(fit_x))]).T
        huber.fit(A, fit_df[f"measured{i}"])
        m, q = huber.coef_[0], huber.intercept_

        linear_fit_force_par.append([f"A{i+1}", m, q])

        ax1 = fig.add_subplot(gs[r, c])
        ax1.scatter(x, query_df[f"measured{i}"], marker="+", color=cmap[0])
        ax1.plot(
            np.arange(x.min(), x.max()),
            np.arange(x.min(), x.max()) * m + q,
            label=f"Fit, m:{m:.2f}",
            color=cmap[-1],
        )
        ax1.axvline(
            xmin, c="green", ls="--", label=f"xmin: {xmin} \nxmax: {xmax}"
        )
        ax1.axvline(xmax, c="green", ls="--")
        ax1.set_xlabel("Encoder position, $\mu$m")
        ax1.set_ylabel("Force, N")
        ax1.set_title(f"A{i+1}(max:{LIMIT_FORCE_TANGENT_CLOSED_LOOP})")
        ax1.legend()

        min, max = (
            query_df[f"measured{i}"].min(),
            query_df[f"measured{i}"].max(),
        )
        if abs(min) > abs(max):
            ax1.axhspan(
                -LIMIT_FORCE_TANGENT_CLOSED_LOOP,
                -LIMIT_FORCE_TANGENT_CLOSED_LOOP,
                color=cmap[-2],
            )
        else:
            ax1.axhspan(
                LIMIT_FORCE_TANGENT_CLOSED_LOOP,
                LIMIT_FORCE_TANGENT_CLOSED_LOOP,
                color=cmap[-2],
            )

        huber.fit(A, fit_df[f"force{i}"])
        m, q = huber.coef_[0], huber.intercept_

        linear_fit_ferror_par.append([f"A{i+1}", m, q])

        ax2 = fig.add_subplot(gs[r, c + 1])
        ax2.scatter(x, query_df[f"force{i}"], marker="+", color=cmap[0])
        ax2.plot(
            np.arange(x.min(), x.max()),
            np.arange(x.min(), x.max()) * m + q,
            label=f"Fit, m:{m:.2f}",
            color=cmap[-1],
        )
        ax2.axvline(
            xmin, c="green", ls="--", label=f"xmin: {xmin} \nxmax: {xmax}"
        )
        ax2.axvline(xmax, c="green", ls="--")
        ax2.set_xlabel("Encoder position, $\mu$m")
        ax2.set_ylabel("Force Error, N")
        ax2.set_title(f"A{i+1}(max:{TANGENT_LINK_LOAD_BEARING_LINK})")
        ax2.legend()

        min, max = query_df[f"force{i}"].min(), query_df[f"force{i}"].max()
        if abs(min) > abs(max):
            ax2.axhspan(
                -TANGENT_LINK_LOAD_BEARING_LINK,
                -TANGENT_LINK_LOAD_BEARING_LINK,
                color=cmap[-2],
            )
        else:
            ax2.axhspan(
                TANGENT_LINK_LOAD_BEARING_LINK,
                TANGENT_LINK_LOAD_BEARING_LINK,
                color=cmap[-2],
            )

        x = query_df[f"position{i+1}"]
        xmin, xmax = x.min(), x.max()

        fit_x = fit_df[f"position{i+1}"]
        A = np.vstack([fit_x, np.zeros(len(fit_x))]).T

        huber.fit(A, fit_df[f"measured{i+1}"])
        m, q = huber.coef_[0], huber.intercept_

        linear_fit_force_par.append([f"A{i+2}", m, q])

        ax3 = fig.add_subplot(gs[r, c + 2])
        ax3.scatter(x, query_df[f"measured{i+1}"], marker="+", color=cmap[0])
        ax3.plot(
            np.arange(x.min(), x.max()),
            np.arange(x.min(), x.max()) * m + q,
            label=f"Fit, m:{m:.2f}",
            color=cmap[-1],
        )
        ax3.axvline(
            xmin, c="green", ls="--", label=f"xmin: {xmin} \nxmax: {xmax}"
        )
        ax3.axvline(xmax, c="green", ls="--")
        ax3.set_xlabel("Encoder position, $\mu$m")
        ax3.set_ylabel("Force, N")
        ax3.set_title(f"A{i+2}(max:{LIMIT_FORCE_TANGENT_CLOSED_LOOP})")
        ax3.legend()

        min, max = (
            query_df[f"measured{i+1}"].min(),
            query_df[f"measured{i+1}"].max(),
        )
        if abs(min) > abs(max):
            ax3.axhspan(
                -LIMIT_FORCE_TANGENT_CLOSED_LOOP,
                -LIMIT_FORCE_TANGENT_CLOSED_LOOP,
                color=cmap[-2],
            )
        else:
            ax3.axhspan(
                LIMIT_FORCE_TANGENT_CLOSED_LOOP,
                LIMIT_FORCE_TANGENT_CLOSED_LOOP,
                color=cmap[-2],
            )

        huber.fit(A, fit_df[f"force{i+1}"])
        m, q = huber.coef_[0], huber.intercept_

        linear_fit_ferror_par.append([f"A{i+2}", m, q])

        ax4 = fig.add_subplot(gs[r, c + 3])
        ax4.scatter(x, query_df[f"force{i+1}"], marker="+", color=cmap[0])
        ax4.plot(
            np.arange(x.min(), x.max()),
            np.arange(x.min(), x.max()) * m + q,
            label=f"Fit, m:{m:.2f}",
            color=cmap[-1],
        )
        ax4.axvline(
            xmin, c="green", ls="--", label=f"xmin: {xmin} \nxmax: {xmax}"
        )
        ax4.axvline(xmax, c="green", ls="--")
        ax4.set_xlabel("Encoder position, $\mu$m")
        ax4.set_ylabel("Force Error, N")
        ax4.set_title(f"A{i+2}(max:{TANGENT_LINK_LOAD_BEARING_LINK})")
        ax4.legend()

        min, max = query_df[f"force{i+1}"].min(), query_df[f"force{i+1}"].max()
        if abs(min) > abs(max):
            ax4.axhspan(
                -TANGENT_LINK_LOAD_BEARING_LINK,
                -TANGENT_LINK_LOAD_BEARING_LINK,
                color=cmap[-2],
            )
        else:
            ax4.axhspan(
                TANGENT_LINK_LOAD_BEARING_LINK,
                TANGENT_LINK_LOAD_BEARING_LINK,
                color=cmap[-2],
            )

        r += 1

    ax5 = fig.add_subplot(gs[-1, :2])
    ax5.plot(x_time, query_df["sum"], color=cmap[0])

    min, max = query_df["sum"].min(), query_df["sum"].max()
    if abs(min) > abs(max):
        ax5.axhspan(
            -TANGENT_LINK_THETA_Z_MOMENT,
            -TANGENT_LINK_THETA_Z_MOMENT,
            color=cmap[-2],
        )
    else:
        ax5.axhspan(
            TANGENT_LINK_THETA_Z_MOMENT,
            TANGENT_LINK_THETA_Z_MOMENT,
            color=cmap[-2],
        )

    ax5.set_xlabel("TimeStamp")
    ax5.set_ylabel("Sum Error, N")
    ax5.set_title(f"Force Error Sum(max:{TANGENT_LINK_THETA_Z_MOMENT})")
    ax5.set_yscale("symlog")
    ax5.set_xticks(ticks)
    ax5.set_xticklabels(ticks_label)

    ax6 = fig.add_subplot(gs[-1, 2:])
    ax6.plot(x_time, query_df.weight, color=cmap[0])

    min, max = query_df[f"weight"].min(), query_df[f"weight"].max()
    if abs(min) > abs(max):
        ax6.axhspan(
            -TANGENT_LINK_TOTAL_WEIGHT_ERROR,
            -TANGENT_LINK_TOTAL_WEIGHT_ERROR,
            color=cmap[-2],
        )
    else:
        ax6.axhspan(
            TANGENT_LINK_TOTAL_WEIGHT_ERROR,
            TANGENT_LINK_TOTAL_WEIGHT_ERROR,
            color=cmap[-2],
        )

    ax6.set_xlabel("TimeStamp")
    ax6.set_ylabel("Weight Error, N")
    ax6.set_title(f"Force Error Weight(max:{TANGENT_LINK_TOTAL_WEIGHT_ERROR})")
    ax6.set_yscale("symlog")
    ax6.set_xticks(ticks)
    ax6.set_xticklabels(ticks_label)

    fig.suptitle(f"{title}\n", fontsize="x-large", fontweight="bold")
    fig.tight_layout()
    # fig.savefig(os.path.join(out_path, f"{prefix}.png"), dpi=300)

    table_force = tabulate.tabulate(
        linear_fit_force_par, headers=["m", "c"], tablefmt="grid"
    )
    table_ferror = tabulate.tabulate(
        linear_fit_ferror_par, headers=["m", "c"], tablefmt="grid"
    )

    with (
        open(
            os.path.join(out_path, f"{prefix}_linear_fit_force.txt"), "w"
        ) as tab_force,
        open(
            os.path.join(out_path, f"{prefix}_linear_fit_ferror.txt"), "w"
        ) as tab_ferror,
    ):
        tab_force.write(
            f"Linear relation: force = encoder_pos*m + c\n{table_force}"
        )
        tab_ferror.write(
            f"Linear relation: force_error = encoder_pos*m + c \n{table_ferror}"
        )

## Relation between the Net displacement and the Force error peak

In [ ]:
def plot_ferror_displacement_relation(
    query_df: pd.DataFrame,
    direction: str,
    title: str,
    cmap: list,
    out_path: str,
    prefix: str,
    huber: HuberRegressor,
) -> None:
    """Function to plot the relation between the displacement and the force error.
    Args:
        query_df (pd.DataFrame): Dataframe with the input data.
        direction (str): Direction of the movement (X, Y or Z).
        title (str): Title of the plot.
        cmap (list): Color map.
        out_path (str): Output path.
        prefix (str): Prefix of the output file.
        huber (HuberRegressor): Huber Regressor object.

    Returns:
        None
    """
    slope = np.abs(np.gradient(query_df[direction]))

    slope = np.where(slope > 0.2, slope, 0.0)

    query_df.loc[:, "slope"] = slope
    query_df.loc[:, "is_moving"] = np.where(query_df.slope != 0, True, False)

    moving_df = query_df[query_df["is_moving"]]
    net_displacement = list()
    net_ferror = {
        0: list(),
        1: list(),
        2: list(),
        3: list(),
        4: list(),
        5: list(),
    }

    init_time = moving_df.index[0]

    threshold = Timedelta(seconds=10.0)

    for i, row in enumerate(moving_df.iterrows()):
        if row[0] - init_time > threshold:
            end_time = moving_df.index[i - 1]
            net_displacement.append(
                abs(
                    moving_df.loc[end_time, direction]
                    - moving_df.loc[init_time, direction]
                )
            )
            for t in range(NUM_TANGENT_LINK):
                min, max = (
                    moving_df.loc[init_time:end_time, f"force{t}"].min(),
                    moving_df.loc[init_time:end_time, f"force{t}"].max(),
                )
                if abs(min) > abs(max):
                    max = abs(min)
                else:
                    max = abs(max)
                net_ferror[t].append(max)
            init_time = row[0]

    A = np.vstack([net_displacement, np.zeros(len(net_displacement))]).T

    fig = plt.figure()
    fig.set_figheight(8)
    fig.set_figwidth(8)

    OLD_LIM = 1000.0
    NEW_LIM = 2000.0

    linear_fit_params = list()
    for key, val in net_ferror.items():
        huber.fit(A, val)
        m, q = huber.coef_[0], huber.intercept_

        l = m * np.array(net_displacement) + q
        chi = np.sqrt(np.sum((val - l) ** 2 / l) / (len(val) - 2))

        linear_fit_params.append([m, q, chi])

        X_OLD_LIM = (OLD_LIM - q) / m
        X_NEW_LIM = (NEW_LIM - q) / m

        index = key + 1
        ax = fig.add_subplot(3, 2, index)
        ax.plot(net_displacement, val, lw=0, marker="+", color=cmap[0])
        ax.plot(
            np.arange(0, X_NEW_LIM),
            m * np.arange(0, X_NEW_LIM) + q,
            ls="-.",
            label=f"fit, m={m:.2f}",  # \n$\chi$:{chi:.2f}',
            color=cmap[1],
        )

        ax.set_xlabel("Net Displacement, $\mu$m")
        ax.set_ylabel("Absolute Force Error, N")
        ax.set_title(f"A{key+1}")

        ax.axhspan(
            1000,
            1000,
            label=f"old lim. x:{int(X_OLD_LIM)}$\mu$m",
            color=cmap[-3],
        )
        ax.axhspan(
            2000,
            2000,
            label=f"new lim. x:{int(X_NEW_LIM)}$\mu$m",
            color=cmap[-1],
        )

        ax.axvspan(X_OLD_LIM, X_OLD_LIM, ls="--", color=cmap[-3])
        ax.axvspan(X_NEW_LIM, X_NEW_LIM, ls="--", color=cmap[-1])

        ax.legend()

    fig.suptitle(f"{title}", fontsize="x-large", fontweight="bold")
    fig.tight_layout()
    # fig.savefig(
    #     os.path.join(out_path, f"{prefix}_relation_displacement_ferror.png"),
    #     dpi=300,
    # )

    table_ferror_displ = tabulate.tabulate(
        linear_fit_params, headers=["m", "c", "$\chi$"], tablefmt="grid"
    )

    with open(
        os.path.join(out_path, f"{prefix}_linear_fit_ferror_displacement.txt"),
        "w",
    ) as tab_ferror:
        tab_ferror.write(
            f"Linear relation: force_error = net_displ*m + c \n{table_ferror_displ}"
        )

## Sync between the movements and the Force Error peaks

In [ ]:
def plot_ferror_movement_timesync(
    query_df: pd.DataFrame,
    direction: str,
    cmap: list,
    prefix: str,
    title: str,
    out_path: str,
) -> None:
    """Function to plot the force error and the movement in the same plot.
    Args:
        query_df (pd.DataFrame): Dataframe with the input data.
        direction (str): Direction of the movement (X, Y or Z).
        cmap (list): Color map.
        prefix (str): Prefix of the output file.
        title (str): Title of the plot.
        out_path (str): Output path.

    Returns:
        None
    """
    fig = plt.figure()
    fig.set_figwidth(8)
    ax = fig.add_subplot(111)
    ax.set_xlabel("TimeStamp")
    ax.set_ylabel("Movememnt, $\mu$m")
    ax2 = ax.twinx()

    x_time = np.arange(len(query_df.index))
    ticks = np.linspace(0, len(query_df.index) - 1, 5)
    ticks_label = [
        query_df.index[int(el)].strftime("%m-%dT%H:%M:%S") for el in ticks
    ]

    axl = ax.plot(x_time, query_df[direction], label="Movement", color=cmap[0])

    lines = axl

    for i in range(NUM_TANGENT_LINK):
        axr = ax2.plot(
            x_time,
            query_df[f"force{i}"],
            label=f"A{i+1}",
            alpha=1.0,
            color=cmap[i + 1],
        )
        lines += axr

    ax2.set_ylabel("Force Error, N")
    labels = [l.get_label() for l in lines]
    ax.legend(lines, labels, loc="upper left", facecolor="#bdbdbd")

    print(ax2.get_zorder(), ax.get_zorder())

    ax.set_xticks(ticks)
    ax.set_xticklabels(ticks_label)
    ax.grid()
    ax2.set_zorder(0)
    ax.set_zorder(1)
    ax.patch.set_visible(False)

    fig.suptitle(f"{title}", fontsize="x-large", fontweight="bold")
    fig.tight_layout()
    # fig.savefig(
    #     os.path.join(out_path, f"{prefix}movement_and_ferror.png"), dpi=150
    # )

## Axial analysis function
This function generates two plots:
* Plot of the mirror walk during the test
* Bar plot with the maximum force reached by each actuator. The Bars are colormapped with the IMS position of the mirror at the moment of maximum force for the respective actuator. The actuator rings are also highlighted in the plot.

In [ ]:
def overview_axial_force(
    query_df: pd.DataFrame,
    direction: str,
    prefix: str,
    title: str,
    out_path: str,
    cmap: list,
) -> None:
    """
    Generate an overview plot of axial force measurements.

    Args:
        query_df (pd.DataFrame): The DataFrame containing the query results.
        direction (str): The direction of movement along the axis.
        prefix (str): The prefix to be used in the output file name.
        title (str): The title of the plot.
        out_path (str): The path to save the output file.
        cmap (list): The colormap to be used in the plot.

    Returns:
        None
    """

    fig = plt.figure()
    fig.set_figheight(8)
    fig.set_figwidth(10)

    x_time = np.arange(len(query_df.index))
    ax = fig.add_subplot(211)
    ax.scatter(x_time, query_df[direction], marker="+", color=cmap[0])
    ax.set_xlabel("TimeStamp")
    ax.set_ylabel(r"Movement, $\mu$m")
    ax.set_title(
        f"Movement Overview along the {direction}-axis",
        fontsize="large",
        fontweight="medium",
    )

    ticks = np.linspace(0, len(query_df.index) - 1, 5)
    ticks_label = [
        query_df.index[int(el)].strftime("%m-%dT%H:%M:%S") for el in ticks
    ]

    ax.set_xticks(ticks)
    ax.set_xticklabels(ticks_label)
    ax.grid()

    x = np.arange(0, NUM_ACTUATOR - NUM_TANGENT_LINK)
    y = np.array(
        [
            np.where(
                abs(query_df[f"measured{i}"].max())
                > abs(query_df[f"measured{i}"].min()),
                (
                    query_df.loc[query_df[f"measured{i}"].idxmax(), "z"],
                    query_df[f"measured{i}"].max(),
                ),
                (
                    query_df.loc[query_df[f"measured{i}"].idxmin(), "z"],
                    query_df[f"measured{i}"].min(),
                ),
            )
            for i in x
        ]
    )

    cmap = mpl.colormaps["viridis"]
    my_cmap = cmap([el / max(y[:, 0]) for el in y[:, 0]])

    sm = cm.ScalarMappable(
        cmap=cmap, norm=plt.Normalize(min(y[:, 0]), max(y[:, 0]))
    )

    ax = fig.add_subplot(212)
    ax.bar(x + 1, y[:, 1], color=my_cmap)
    ax.set_title(
        f"Axial actuator Max force measured",
        fontsize="large",
        fontweight="medium",
    )

    ax.axhline(444, color="red")
    ax.annotate("CL Force Limit", (-2, 425), color="red")
    ax.axvspan(0, 29.40, edgecolor="blue", fill=False)
    ax.annotate("B-ring", (29.5 / 2, 400), color="blue")
    ax.axvspan(29.60, 53.40, edgecolor="green", fill=False)
    ax.annotate("C-ring", (29.5 + (53.5 - 29.5) / 2, 400), color="green")
    ax.axvspan(53.60, 72.5, edgecolor="orange", fill=False)
    ax.annotate("D-ring", (53.5 + (72.5 - 53.5) / 2, 400), color="orange")
    ax.set_xlabel("Id. Actuator")
    ax.set_ylabel("Max Measured Force, N")

    cbar = plt.colorbar(sm, ax=ax)
    cbar.set_label("IMS position")

    fig.suptitle(f"{title}\n", fontsize="x-large", fontweight="bold")
    fig.tight_layout()
    # fig.savefig(os.path.join(out_path, f"{prefix}_axial.png"), dpi=300)

## Global variables

In [ ]:
topics = [
    "lsst.sal.MTM2.tangentEncoderPositions",
    "lsst.sal.MTM2.tangentForce",
    "lsst.sal.MTM2.forceErrorTangent",
    "lsst.sal.MTM2.netForcesTotal",
    "lsst.sal.MTM2.netMomentsTotal",
    "lsst.sal.MTM2.positionIMS",
    "lsst.sal.MTM2.displacementSensors",
    "lsst.sal.MTM2.zenithAngle",
]

huber = HuberRegressor(epsilon=1.7)
tol = Timedelta(seconds=1.0)
cmap = [
    "#b35806",
    "#e08214",
    "#fdb863",
    "#fee0b6",
    "#f7f7f7",
    "#d8daeb",
    "#b2abd2",
    "#8073ac",
    "#542788",
]

efd_client = lsst_efd_client.EfdClient(efd_name="usdf_efd")

# TMA

## X-axis

In [ ]:
# first dataframe

# x (test multi-range)
start_time = Time("2023-11-27T16:55:00.000", format="isot")
end_time = Time("2023-11-27T18:55:00.000", format="isot")


# start_time = Time("2024-02-27T00:15:00.000", format="isot")
# end_time = Time("2024-02-27T00:50:00.000", format="isot")


query_df1 = await get_merge_query(
    efd_client=efd_client,
    topics=topics,
    start_time=start_time,
    end_time=end_time,
    tolerance=tol,
)

# # x (test long-range)
start_time = Time("2024-02-27T00:00:00.000", format="isot")
end_time = Time("2024-02-27T01:00:00.000", format="isot")

query_df2 = await get_merge_query(
    efd_client=efd_client,
    topics=topics,
    start_time=start_time,
    end_time=end_time,
    tolerance=tol,
)

query_df = pd.concat([query_df2, query_df1])
query_df.sort_index(inplace=True)

prefix = "TMA_x_tolimits-multirange-zenith-new"
direction = prefix.split("_")[1][0]
title = f"{prefix.split('_')[0]} {prefix.split('_')[1]}"
out_path = os.path.join(prefix)
if not os.path.exists(out_path):
    os.mkdir(out_path)

plot_overview(
    query_df=query_df,
    direction=direction,
    prefix=prefix,
    title=title,
    out_path=out_path,
    huber=huber,
    cmap=cmap,
)

plot_ferror_displacement_relation(
    query_df=query_df,
    direction=direction,
    title=title,
    cmap=cmap,
    out_path=out_path,
    prefix=prefix,
    huber=huber,
)

plot_ferror_movement_timesync(
    query_df=query_df,
    direction=direction,
    cmap=cmap,
    prefix=prefix,
    title=title,
    out_path=out_path,
)

## Y-axis

In [ ]:
# first dataframe

# y (test multi-range)
start_time = Time("2023-11-27T18:50:00.000", format="isot")
end_time = Time("2023-11-27T20:18:00.000", format="isot")


query_df1 = await get_merge_query(
    efd_client=efd_client,
    topics=topics,
    start_time=start_time,
    end_time=end_time,
    tolerance=tol,
)


# # y (test long-range)
start_time = Time("2023-11-28T16:50:00.000", format="isot")
end_time = Time("2023-11-28T18:00:00.000", format="isot")

query_df2 = await get_merge_query(
    efd_client=efd_client,
    topics=topics,
    start_time=start_time,
    end_time=end_time,
    tolerance=tol,
)

query_df = pd.concat([query_df2, query_df1])
query_df.sort_index(inplace=True)

prefix = "TMA_y_tolimits-multirange-zenith-new"
direction = prefix.split("_")[1][0]
title = f"{prefix.split('_')[0]} {prefix.split('_')[1]}"
out_path = os.path.join(prefix)
if not os.path.exists(out_path):
    os.mkdir(out_path)

plot_overview(
    query_df=query_df1,
    direction=direction,
    prefix=prefix,
    title=title,
    out_path=out_path,
    huber=huber,
    cmap=cmap,
)

plot_ferror_displacement_relation(
    query_df=query_df1,
    direction=direction,
    title=title,
    cmap=cmap,
    out_path=out_path,
    prefix=prefix,
    huber=huber,
)

plot_ferror_movement_timesync(
    query_df=query_df1,
    direction=direction,
    cmap=cmap,
    prefix=prefix,
    title=title,
    out_path=out_path,
)

## Z-axis

In [ ]:
start_time = Time("2023-11-27T14:20:00.000", format="isot")
end_time = Time("2023-11-27T15:33:00.000", format="isot")

query_df = await get_merge_query(
    efd_client=efd_client,
    topics=["lsst.sal.MTM2.axialForce", "lsst.sal.MTM2.positionIMS"],
    start_time=start_time,
    end_time=end_time,
    tolerance=tol,
)

prefix = "TMA_z"
direction = prefix.split("_")[1][0]
title = f"{prefix.split('_')[0]} {prefix.split('_')[1]}"
out_path = os.path.join(prefix)
if not os.path.exists(out_path):
    os.mkdir(out_path)

overview_axial_force(
    query_df=query_df,
    direction=direction,
    prefix=prefix,
    title=title,
    out_path=out_path,
    cmap=cmap,
)

## Level 3

In [ ]:
efd_client = lsst_efd_client.EfdClient(efd_name="idf_efd")

## FACING DOWNWARD

### X-axis

In [ ]:
start_time = Time("2023-06-28T23:35:00.000", format="isot")
end_time = Time("2023-06-28T23:57:00.000", format="isot")

query_df = await get_merge_query(
    efd_client=efd_client,
    topics=topics,
    start_time=start_time,
    end_time=end_time,
    tolerance=tol,
)

query_df.sort_index(inplace=True)
query_df.dropna(inplace=True)

prefix = "Level3_x"
direction = prefix.split("_")[1][0]
title = f"{prefix.split('_')[0]} {prefix.split('_')[1]}"
out_path = os.path.join(prefix)
if not os.path.exists(out_path):
    os.mkdir(out_path)

plot_overview(
    query_df=query_df,
    direction=direction,
    prefix=prefix,
    title=title,
    out_path=out_path,
    huber=huber,
    cmap=cmap,
)

plot_ferror_displacement_relation(
    query_df=query_df,
    direction=direction,
    title=title,
    cmap=cmap,
    out_path=out_path,
    prefix=prefix,
    huber=huber,
)

plot_ferror_movement_timesync(
    query_df=query_df,
    direction=direction,
    cmap=cmap,
    prefix=prefix,
    title=title,
    out_path=out_path,
)

### Y-axis

In [ ]:
start_time = Time("2023-06-29T00:02:00.000", format="isot")
end_time = Time("2023-06-29T00:30:00.000", format="isot")


query_df = await get_merge_query(
    efd_client=efd_client,
    topics=topics,
    start_time=start_time,
    end_time=end_time,
    tolerance=tol,
)

query_df.sort_index(inplace=True)
query_df.dropna(inplace=True)

prefix = "Level3_y"
direction = prefix.split("_")[1][0]
title = f"{prefix.split('_')[0]} {prefix.split('_')[1]}"
out_path = os.path.join(prefix)
if not os.path.exists(out_path):
    os.mkdir(out_path)

plot_overview(
    query_df=query_df,
    direction=direction,
    prefix=prefix,
    title=title,
    out_path=out_path,
    huber=huber,
    cmap=cmap,
)

plot_ferror_displacement_relation(
    query_df=query_df,
    direction=direction,
    title=title,
    cmap=cmap,
    out_path=out_path,
    prefix=prefix,
    huber=huber,
)

plot_ferror_movement_timesync(
    query_df=query_df,
    direction=direction,
    cmap=cmap,
    prefix=prefix,
    title=title,
    out_path=out_path,
)

### Z-axis

In [ ]:
start_time = Time("2023-06-29T00:34:00.000", format="isot")
end_time = Time("2023-06-29T01:10:00.000", format="isot")


query_df = await get_merge_query(
    efd_client=efd_client,
    topics=["lsst.sal.MTM2.axialForce", "lsst.sal.MTM2.positionIMS"],
    start_time=start_time,
    end_time=end_time,
    tolerance=tol,
)

prefix = "level3_z"
direction = prefix.split("_")[1][0]
title = f"{prefix.split('_')[0]} {prefix.split('_')[1]}"
out_path = os.path.join(prefix)
if not os.path.exists(out_path):
    os.mkdir(out_path)

overview_axial_force(
    query_df=query_df,
    direction=direction,
    prefix=prefix,
    title=title,
    out_path=out_path,
    cmap=cmap,
)

## HORIZON

### X-axis

In [ ]:
start_time = Time("2023-06-29T02:20:00.000", format="isot")
end_time = Time("2023-06-29T02:45:00.000", format="isot")

query_df = await get_merge_query(
    efd_client=efd_client,
    topics=topics,
    start_time=start_time,
    end_time=end_time,
    tolerance=tol,
)

query_df.sort_index(inplace=True)
query_df.dropna(inplace=True)

prefix = "Level3_x_horizon"
direction = prefix.split("_")[1][0]
title = f"{prefix.split('_')[0]} {prefix.split('_')[1]}"
out_path = os.path.join(prefix)
if not os.path.exists(out_path):
    os.mkdir(out_path)

plot_overview(
    query_df=query_df,
    direction=direction,
    prefix=prefix,
    title=title,
    out_path=out_path,
    huber=huber,
    cmap=cmap,
)

plot_ferror_displacement_relation(
    query_df=query_df,
    direction=direction,
    title=title,
    cmap=cmap,
    out_path=out_path,
    prefix=prefix,
    huber=huber,
)

plot_ferror_movement_timesync(
    query_df=query_df,
    direction=direction,
    cmap=cmap,
    prefix=prefix,
    title=title,
    out_path=out_path,
)

### Y-axis

In [ ]:
start_time = Time("2023-06-29T03:25:00.000", format="isot")
end_time = Time("2023-06-29T03:37:00.000", format="isot")

query_df = await get_merge_query(
    efd_client=efd_client,
    topics=topics,
    start_time=start_time,
    end_time=end_time,
    tolerance=tol,
)

query_df.sort_index(inplace=True)
query_df.dropna(inplace=True)

prefix = "Level3_y_horizon"
direction = prefix.split("_")[1][0]
title = f"{prefix.split('_')[0]} {prefix.split('_')[1]}"
out_path = os.path.join(prefix)
if not os.path.exists(out_path):
    os.mkdir(out_path)

plot_overview(
    query_df=query_df,
    direction=direction,
    prefix=prefix,
    title=title,
    out_path=out_path,
    huber=huber,
    cmap=cmap,
)

plot_ferror_displacement_relation(
    query_df=query_df,
    direction=direction,
    title=title,
    cmap=cmap,
    out_path=out_path,
    prefix=prefix,
    huber=huber,
)

plot_ferror_movement_timesync(
    query_df=query_df,
    direction=direction,
    cmap=cmap,
    prefix=prefix,
    title=title,
    out_path=out_path,
)

### Z-axis

In [ ]:
start_time = Time("2023-06-29T01:55:00.000", format="isot")
end_time = Time("2023-06-29T02:17:00.000", format="isot")


query_df = await get_merge_query(
    efd_client=efd_client,
    topics=["lsst.sal.MTM2.axialForce", "lsst.sal.MTM2.positionIMS"],
    start_time=start_time,
    end_time=end_time,
    tolerance=tol,
)

prefix = "level3_z_horizon"
direction = prefix.split("_")[1][0]
title = f"{prefix.split('_')[0]} {prefix.split('_')[1]}"
out_path = os.path.join(prefix)
if not os.path.exists(out_path):
    os.mkdir(out_path)

overview_axial_force(
    query_df=query_df,
    direction=direction,
    prefix=prefix,
    title=title,
    out_path=out_path,
    cmap=cmap,
)

# ISSUE FIXED
As reported in [OBS-300](https://rubinobs.atlassian.net/browse/OBS-300) and [DM-41855](https://rubinobs.atlassian.net/browse/DM-41855#icft=DM-41855), the problem come from a wrong behaviour of the tangent link, all acting as master. After changing the system software to slaves the tangent links to the hardpoints, the tangent force error peaks are disappeard now, as one can see from the repetition of the RBM test presented in the folowing cells

## X-AXIS

In [ ]:
start_time = Time("2024-02-27T00:15:00.000", format="isot")
end_time = Time("2024-02-27T00:50:00.000", format="isot")

query_df = await get_merge_query(
    efd_client=efd_client,
    topics=topics,
    start_time=start_time,
    end_time=end_time,
    tolerance=tol,
)

prefix = "TMA_x_fixed"
direction = prefix.split("_")[1][0]
title = f"{prefix.split('_')[0]} {prefix.split('_')[1]}"
out_path = os.path.join(prefix)
if not os.path.exists(out_path):
    os.mkdir(out_path)

plot_overview(
    query_df=query_df,
    direction=direction,
    prefix=prefix,
    title=title,
    out_path=out_path,
    huber=huber,
    cmap=cmap,
)

plot_ferror_displacement_relation(
    query_df=query_df,
    direction=direction,
    title=title,
    cmap=cmap,
    out_path=out_path,
    prefix=prefix,
    huber=huber,
)

plot_ferror_movement_timesync(
    query_df=query_df,
    direction=direction,
    cmap=cmap,
    prefix=prefix,
    title=title,
    out_path=out_path,
)

## Y-AXIS

In [ ]:
start_time = Time("2024-02-27T00:45:00.000", format="isot")
end_time = Time("2024-02-27T01:00:00.000", format="isot")

query_df = await get_merge_query(
    efd_client=efd_client,
    topics=topics,
    start_time=start_time,
    end_time=end_time,
    tolerance=tol,
)

prefix = "TMA_y_fixed"
direction = prefix.split("_")[1][0]
title = f"{prefix.split('_')[0]} {prefix.split('_')[1]}"
out_path = os.path.join(prefix)
if not os.path.exists(out_path):
    os.mkdir(out_path)

plot_overview(
    query_df=query_df,
    direction=direction,
    prefix=prefix,
    title=title,
    out_path=out_path,
    huber=huber,
    cmap=cmap,
)

plot_ferror_displacement_relation(
    query_df=query_df,
    direction=direction,
    title=title,
    cmap=cmap,
    out_path=out_path,
    prefix=prefix,
    huber=huber,
)

plot_ferror_movement_timesync(
    query_df=query_df,
    direction=direction,
    cmap=cmap,
    prefix=prefix,
    title=title,
    out_path=out_path,
)